In [1]:
import torch
import torchinfo
from torch.utils.data import DataLoader
import timm  # Import the timm library to access the Xception model
import os

from level_1_dataloader import image_dataloader
from model.level_1_classifier import level_1_classifier, level_1_model, level_1_output_layer
from model.level_2_classifier import level_2_classifier, level_2_output_layer, level_2_pre_model_concate, level_2_post_model
from Trainer_1 import Trainer_level_1, Trainer_level_2
from Tester_1 import Tester_level_1, Tester_level_2

import copy


C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_set = image_dataloader(csv_file = 'train_set.csv', root_dir = 'coverpage/coverpage/')
val_set = image_dataloader(csv_file = 'val_set.csv', root_dir = 'coverpage/coverpage/')
test_set = image_dataloader(csv_file = 'test_set.csv', root_dir = 'coverpage/coverpage/')

In [3]:
print('no. of training sample',len(train_set))
print('no. of validation sample',len(val_set))
print('no. of testing sample',len(test_set))

no. of training sample 22340
no. of validation sample 2804
no. of testing sample 2809


In [4]:

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
print(len(train_dataloader))

val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
print(len(val_dataloader))

test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)
print(len(test_dataloader))

699
88
88


In [5]:
def assign_gpu_model(model):
    print("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return model

In [8]:
import torch
import torchvision.models as models

In [9]:
def fine_tune_load_model():
    """Loads pre-trained DenseNet-121 model and removes the classification head."""
    # Load the pre-trained DenseNet-121 model
    model = models.densenet121(pretrained=True)
    
    print("Original DenseNet-121 model:")
    print(model)

    # Remove the classification head
    # DenseNet's classifier is typically called 'classifier' (a Linear layer)
    # We will replace the classifier with an identity operation
    num_features = model.classifier.in_features
    model.classifier = torch.nn.Identity()

    # Wrap the model to include a Flatten layer after the features extraction
    class DenseNet121Features(torch.nn.Module):
        def __init__(self, model):
            super(DenseNet121Features, self).__init__()
            self.features = model.features
            self.flatten = torch.nn.Flatten()
        
        def forward(self, x):
            x = self.features(x)
            x = torch.nn.functional.relu(x, inplace=True)
            x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))
            x = self.flatten(x)
            return x

    # Create the modified model
    new_model = DenseNet121Features(model)

    return new_model

In [10]:
FE_model = fine_tune_load_model()

Original DenseNet-121 model:
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_runn

C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
torchinfo.summary(FE_model, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
DenseNet121Features                      [1, 3, 224, 224]          [1, 1024]                 --                        --                        --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 1024, 7, 7]           --                        --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
│    └─ReLU: 2-3                         [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --                        --
│    └─MaxPool2d: 2-4                    [1, 64, 112, 112]         [1, 64, 56, 56]           --

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
DenseNet121Features                      [1, 3, 224, 224]          [1, 1024]                 --                        --                        --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 1024, 7, 7]           --                        --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
│    └─ReLU: 2-3                         [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --                        --
│    └─MaxPool2d: 2-4                    [1, 64, 112, 112]         [1, 64, 56, 56]           --

In [12]:
import os

# Given variables
used_model = 'DenseNet-121'
used_model_feature_size = 1024  # Assuming ResNet-101's output feature size is 2048
level_1_checkpoint_dir = './checkpoints/'+used_model+'/level_1/'
level_1_model_file = 'model.pth'
level_1_csvlogger_file = 'log.csv'
level_1_weights_path = os.path.join(level_1_checkpoint_dir, level_1_model_file)
feature_size_extract_from_level_1 = 64  # Update this value as needed
no_epoch = 10
lr = 0.0001
optimizer = 'AdamW'

# Define the new directory path for level 1
new_checkpoint_dir = 'Downloads/checkpoints'
level_1_checkpoint_dir = os.path.join(new_checkpoint_dir, used_model, 'level_1')
os.makedirs(level_1_checkpoint_dir, exist_ok=True)

# Update other paths that depend on level_1_checkpoint_dir
level_1_weights_path = os.path.join(level_1_checkpoint_dir, level_1_model_file)


In [13]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)

In [14]:

# print(model_level_1)
torchinfo.summary(model_level_1, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
level_1_model                                 [1, 3, 224, 224]          [1, 2]                    --                        --                        --
├─DenseNet121Features: 1-1                    [1, 3, 224, 224]          [1, 1024]                 --                        --                        --
│    └─Sequential: 2-1                        [1, 3, 224, 224]          [1, 1024, 7, 7]           --                        --                        --
│    │    └─Conv2d: 3-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
│    │    └─BatchNorm2d: 3-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
│    │    └─ReLU: 3-3                         [1, 64, 112, 112]  

C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
level_1_model                                 [1, 3, 224, 224]          [1, 2]                    --                        --                        --
├─DenseNet121Features: 1-1                    [1, 3, 224, 224]          [1, 1024]                 --                        --                        --
│    └─Sequential: 2-1                        [1, 3, 224, 224]          [1, 1024, 7, 7]           --                        --                        --
│    │    └─Conv2d: 3-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
│    │    └─BatchNorm2d: 3-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
│    │    └─ReLU: 3-3                         [1, 64, 112, 112]  

In [15]:
trainer = Trainer_level_1(model=model_level_1, level=1, training_dataloader=train_dataloader, validation_dataloader=val_dataloader, epoch=no_epoch, learning_rate=lr, use_gpu=True, opt_method=optimizer, checkpoint_dir=level_1_checkpoint_dir, checkpoint_filename=level_1_model_file, csv_logger=level_1_csvlogger_file)


In [16]:
trainer.run()

Finish initializing...
2024-06-07 15:36:49.718806
EPOCH 1:


  0%|                                                                                       | 0/699 [00:02<?, ?batch/s]


KeyboardInterrupt: 

In [ ]:
# from config.Tester import Tester
tester = Tester_level_1(model = model_level_1, weights_file = level_1_model_file, level =1,  data_loader = test_dataloader, checkpoint_dir = level_1_checkpoint_dir)

In [ ]:
tester.prediction()

In [ ]:
no_epoch = 100
lr = 0.001
optimizer = 'AdamW'
# Define the base checkpoint directory
level_2_checkpoint_dir = './checkpoints/' + used_model + '/level_2_'

# Create directories for fiction and non-fiction checkpoints
fiction_checkpoint_dir = os.path.join(level_2_checkpoint_dir, 'fiction')
non_fiction_checkpoint_dir = os.path.join(level_2_checkpoint_dir, 'non_fiction')

# Ensure directories exist (avoid errors if not created beforehand)
os.makedirs(fiction_checkpoint_dir, exist_ok=True)
os.makedirs(non_fiction_checkpoint_dir, exist_ok=True)

fiction_gamma_pos = 4
fiction_gamma_neg = 5
non_fiction_gamma_pos = 4
non_fiction_gamma_neg = 5

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

In [ ]:
def freeze_used_model(model):
    
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())[:-2]
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

def freeze_level_1_model(model):
    
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())[:-1]
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

def freeze_full_model(model):
    
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

In [ ]:
used_model = freeze_used_model(copy.deepcopy(model_level_1))

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

FE_model_level_1 = freeze_level_1_model(copy.deepcopy(model_level_1))

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

model_level_1 = freeze_full_model(copy.deepcopy(model_level_1))

In [ ]:
torchinfo.summary(used_model, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

In [ ]:
torchinfo.summary(FE_model_level_1, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

In [ ]:
torchinfo.summary(model_level_1, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

In [ ]:
# Level 2 classifier
fiction_classifier = level_2_classifier(model_feature_size=used_model_feature_size, feature_from_level_1=feature_size_extract_from_level_1)
fiction_op_layer = level_2_output_layer(feature_size_extract_from_level_2=64, no_class=29)
fiction_model = level_2_post_model(level_2_classifier=fiction_classifier, level_2_output_layer=fiction_op_layer)

In [ ]:
non_fiction_classifier = level_2_classifier(model_feature_size=used_model_feature_size, feature_from_level_1=feature_size_extract_from_level_1)
non_fiction_op_layer = level_2_output_layer(feature_size_extract_from_level_2=64, no_class=29)
non_fiction_model = level_2_post_model(level_2_classifier=non_fiction_classifier, level_2_output_layer=non_fiction_op_layer)

In [ ]:
level_2_pre_model = level_2_pre_model_concate(used_model, FE_model_level_1)

In [ ]:
level_2_pre_model

In [ ]:
level_2_trainer = Trainer_level_2(
    level_1_model=model_level_1,
    level_2_pre_model=level_2_pre_model,
    fiction_model=fiction_model,
    non_fiction_model=non_fiction_model,
    training_dataloader=train_dataloader,
    validation_dataloader=val_dataloader,
    epoch=no_epoch,
    learning_rate=lr,
    use_gpu=True,
    opt_method=optimizer,
    fiction_checkpoint_dir=fiction_checkpoint_dir,
    non_fiction_checkpoint_dir=non_fiction_checkpoint_dir,
    checkpoint_filename=level_1_model_file,
    fiction_gamma_pos=fiction_gamma_pos,
    fiction_gamma_neg=fiction_gamma_neg,
    non_fiction_gamma_pos=non_fiction_gamma_pos,
    non_fiction_gamma_neg=non_fiction_gamma_neg,
    csv_logger=level_1_csvlogger_file,
)

In [ ]:
level_2_trainer.run()

In [ ]:
from config.Tester import Tester_level_2

In [ ]:
tester_level_2 = Tester_level_2(level_1_model = model_level_1, level_2_pre_model = level_2_pre_model, fiction_model =  fiction_model, non_fiction_model = non_fiction_model, weights_file = level_1_model_file, data_loader = test_dataloader, fiction_checkpoint_dir = level_2_checkpoint_dir+'fiction', non_fiction_checkpoint_dir = level_2_checkpoint_dir+'non_fiction', fiction_gamma_pos = fiction_gamma_pos, fiction_gamma_neg = fiction_gamma_neg, non_fiction_gamma_pos = non_fiction_gamma_pos, non_fiction_gamma_neg = non_fiction_gamma_neg)

In [ ]:
tester_level_2.prediction()